# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [4]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,32,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.dense = nn.Sequential(
            nn.Linear(32*14*14,128),
            nn.ReLU(),
            nn.Linear(128,10))
        
    def forward(self,x):
        conv_out = self.conv(x)
        res = conv_out.view(conv_out.size(0),-1)
        out = self.dense(res)
        return out
    
   
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    # training-----------------------------
    start_time = time.time()
    train_loss = 0.
    train_acc = 0.
    for images, labels in tqdm(train_loader):
        images = torch.autograd.Variable(images)
        labels = torch.autograd.Variable(labels)
        out = model(images)
        loss = criterion(out, labels)
        train_loss += loss.data.item()*labels.size(0)
        pred = torch.max(out, 1)[1]
        train_correct = (pred == labels).sum()
        train_acc += train_correct.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    stop_time = time.time()
    print('| train loss:%f | train acc:%f | train time:%f |'%
          (train_loss/len(train_dataset),train_acc/len(train_dataset),(stop_time-start_time)))

    # evaluation--------------------------------
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for images, labels in tqdm(test_loader):
        images = torch.autograd.Variable(images, requires_grad=False)
        labels = torch.autograd.Variable(labels, requires_grad=False)
        out = model(images)
        loss = criterion(out, labels)
        eval_loss += loss.data.item()*labels.size(0)
        pred = torch.max(out, 1)[1]
        num_correct = (pred == labels).sum()
        eval_acc += num_correct.data.item()
    print('|test loss: {:.6f} | test acc: {:.6f}|'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
    


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [01:09<00:00,  6.86it/s]


| train loss:0.302944 | train acc:0.913700 | train time:69.752243 |


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:04<00:00, 15.93it/s]


|test loss: 0.106830 | test acc: 0.968900|


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [01:18<00:00,  6.19it/s]


| train loss:0.089300 | train acc:0.972150 | train time:78.062204 |


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:05<00:00, 15.21it/s]


|test loss: 0.073923 | test acc: 0.975600|


  4%|███▎                                                                             | 19/468 [00:03<01:16,  5.84it/s]


KeyboardInterrupt: 

#### Q5:
Please print the training and testing accuracy.